# Tweets from bioRxiv and medRxiv

Find and display events from the CSHL DOI prefix 10.1101 from the last 7 days, and find the most tweeted preprints.

In [1]:
import sys
sys.path.append('../')
sys.path.insert(0, '..')

import pandas as pd # data analysis library
import json
import mrced2 # module to run event data queries
import os # some file manipulations
import math # some number manipulations
import altair.vegalite.v3 as alt # some data visualizations
from IPython.display import Markdown as md # some markdown manipulations
from datetime import datetime, date, timedelta # some date manipulations

In [2]:
email = "info@front-matter.io"
prefix = "10.1101"
start_date = (date.today() - timedelta(days = 7)).strftime('%Y-%m-%d')
end_date = date.today()

In [3]:
ed = mrced2.eventData(mailto = email)
ed.buildQuery({'obj-id.prefix' : prefix, 'source': 'twitter', 'rows': 500,'from-occurred-date' : start_date, 'until-occurred-date' : end_date})

https://api.eventdata.crossref.org/v1/events?mailto=info@front-matter.io&obj-id.prefix=10.1101&source=twitter&rows=500&from-occurred-date=2021-09-27&until-occurred-date=2021-10-04


In [4]:
ed.runQuery(retry = 5)

Event Data query started...
API query complete  200
output file written to 1101/tweets.json


In [5]:
pages = math.ceil(ed.events.getHits() / 1000)

0 events found


In [6]:
email = "info@front-matter.io"
prefix = "10.1101"
start_date = date.today() - timedelta(days = 7)
end_date = date.today()

# find the all result pages for the search
ed = mrced2.eventData(mailto = email)
ed.getAllPages(pages, {'rows': 1000, 'obj-id.prefix' : prefix, 'source': 'twitter', 'from-occurred-date' : start_date, 'until-occurred-date' : end_date}, fileprefix = '1101/tweets_') 

Initialisation to look at the properties of the results.

In [7]:
jd1 = mrced2.eventRecord() # instance of a class to interpret the events
files = os.listdir('1101') # get all the filenames

jd1.mergeJsons(files, folder = '1101') # load the json event data from multiple files

failed to load preprint_tweets_2021-07-29.csv
failed to load preprint_tweets_2021-09-06.csv
failed to load preprint_tweets_2021-09-13.csv
failed to load .DS_Store
failed to load .gitkeep
failed to load preprint_tweets_2021-08-30.csv
failed to load preprint_tweets_2021-08-09.csv
failed to load preprint_tweets_2021-08-23.csv
failed to load preprint_tweets_2021-08-02.csv
failed to load preprint_tweets_2021-08-16.csv
failed to load .ipynb_checkpoints
failed to load preprint_tweets_2021-07-26.csv
failed to load preprint_tweets_2021-09-20.csv
output file written to 1101/tweets.json


In [8]:
js = json.load(open("1101/tweets.json"))
df = pd.json_normalize(js, record_path = ['message', 'events'])
gdf = df.groupby(['obj_id']).size().reset_index(name='count').sort_values('count', ascending=False)
cdf = gdf[gdf['count'] >= 3]

In [9]:
email = "info@front-matter.io"
rest = mrced2.restApi(email = email)

data = []
for index, row in cdf.iterrows():
    rest.runQuery(row)
    if rest.work is not None and date.fromisoformat(rest.work["posted"]) >= start_date:
        data.append(rest.work)
print(data)
    
tdf = pd.DataFrame(data, columns=['doi','tweets','archive','subject-area','covid','title','authors','abstract','posted'])
tdf.to_csv('1101/preprint_tweets_' + str(date.today().strftime('%Y-%m-%d')) + '.csv')

tdf.head(50)

REST API query started for 10.1101/2021.07.13.452194...
REST API query complete  200
REST API query started for 10.1101/2021.07.31.21261387...
REST API query complete  200
REST API query started for 10.1101/2021.07.08.21260210...
REST API query complete  200
REST API query started for 10.1101/2021.04.19.21255739...
REST API query complete  200
REST API query started for 10.1101/2021.06.01.21258176...
REST API query complete  200
REST API query started for 10.1101/2021.04.20.21255670...
REST API query complete  200
REST API query started for 10.1101/2021.06.20.21259209...
REST API query complete  200
REST API query started for 10.1101/2021.04.15.21252192...
REST API query complete  200
REST API query started for 10.1101/2021.08.17.21262169...
REST API query complete  200
REST API query started for 10.1101/2021.05.03.21256520...
REST API query complete  200
REST API query started for 10.1101/2021.06.11.21258690...
REST API query complete  200
REST API query started for 10.1101/2021.08.08

,doi,tweets,archive,subject-area,covid,title,authors,abstract,posted


### Tweets of bioRxiv and medRxiv preprints

In [10]:
num_rows = tdf['archive'].count()
num_covid = tdf['covid'].value_counts(ascending=True)[1]
num_biorxiv = tdf['archive'].value_counts(ascending=True)[1]
num_medrxiv = tdf['archive'].value_counts(ascending=True)[0]
end_date = date.today().strftime('%Y-%m-%d')
max_count = tdf['tweets'].max()

md('{} preprints (including {} covering SARS-CoV-2, {} from bioRxiv and {} from medRxiv) published in the last 7 days before {} had been tweeted at least 3 times (maximum {}).'.format(num_rows, num_covid, num_biorxiv, num_medrxiv, end_date, max_count))

IndexError: index 1 is out of bounds for axis 0 with size 0